In [6]:
import torch
import numpy as np
from torch.nn import GRU
from torch_geometric.data import Data

In [7]:
class TFG:
    def __init__(self, data):
        self.data = data
        size = self.data.x.shape[1]
        self.word_bigru = GRU(size, size, bidirectional=True)
        self.relationship_bigru = GRU(size, size, bidirectional=True)
        
    def train(self):
        middle_node = dict()
        relationship_map = dict()
        
        # assume retrieve relationship data
        edge_index = self.data.edge_index.numpy()
        for i, node_type in enumerate(self.data.y.numpy()):
            if node_type == 1:
                if i in middle_node:
                    relationship_map[middle_node[i]] = edge_index[i][1]
                else:
                    middle_node[edge_index[i][1]] = i
        
        # convert relationship data to be Tensor
        edge_attr = self.data.edge_attr.numpy()
        relationship_vector = []
        for key, value in relationship_map.items():
            relationship_vector.append([edge_attr[key], edge_attr[value]])
            
        relationships = torch.Tensor(relationship_vector)
        
        # run two kinds of bigru
        out1, hidden1 = words_bigru(self.data.x)
        out2, hidden2 = relationship_bigru(relationships)
        
        data = Data(x=hidden1, y=self.data.y, edge_index=self.data.edge_index, edge_attr=hidden2)
        return data